# reminder
- offset (initial margin) (add something)
- age, gender, 
- build an initial model based on age, gender, etc
    - can add weight and height, but as a quadratic 
    - e.g., GLM
- add the output of the initial model as a feature to the dlr
    - as an offset, not as a feature (offset is a parameter?)

# loading packages

In [ ]:
import sys
import re
sys.path.append("./ds_utils") # for reading self-de fined modules
from ds_utils.ds_preamble import *  # # self-defined module
from ds_utils.ds_helper import *  # self-defined module
from ds_utils.ds_plotting import *  # # self-defined module
from math import ceil 
# from smart_open import open 

import dsar # custom library to read data from cloud

pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 100)
pd.set_option('display.precision', 5)


timestamp = get_timestamp() # this timestamp will be appended to many output files
random_state=1234567 # used to make results reproducible

use_expo_less_than_thresh = False
bmi_offset = True
# bmi_offset = False

# cohort_include_smoker=False
# formulation_name ='f2'
# use_adj_premium = True
# use_inflate=False

# remap_model = {'score_lr_xgb': 'dlr'}
remap_model = False

# ------------------------------- refit using R ------------------------------ #
# lr_out = 'refit_hb_lr_output_20220714_163622' # SELECT * FROM temp.refit_hb_lr_output_20220714_163622
# lr_out = 'refit_hb_lr_output_sim_m238'
# pred_col_names = ['score_lr_xgb']
# ---------------------------------------------------------------------------- #


# ------------------------ important variations to run ----------------------- #
# lr_out = 'hb_lr_output_20220714_163622' # with weight height, n_estimator = 20
# lr_out = 'hb_lr_output_20220715_114029' # no weight height, n_estimator = 120
# lr_out = 'hb_lr_output_20220726_154536' # keep app_age>=6; no weight height
# lr_out = 'hb_lr_output_20220726_173905' # keep app_age>=6; with weight height
# lr_out = 'hb_lr_output_20220727_135834' # keep app_age>=18; no weight height
# lr_out = 'hb_lr_output_20220727_135726' # keep app_age>=18; with weight height
# ---------------------------------------------------------------------------- #


# ---------------------- old format, before final report --------------------- #
# lr_out = 'hb_lr_output_comb_allyr_loss_ratio_20220525_162627'
# pred_col_names = ['xgb', 'freq_severity', 'cla_reg']

# -------------------------------- new format -------------------------------- #

# ------------------------ with test data in train set ----------------------- #
# lr_out = 'hb_lr_output_20220715_114029' # no weight height, n_estimator = 120
# lr_out = 'hb_lr_output_20220714_163400' # no weight height, n_estimator = 150
# lr_out = 'hb_lr_output_20220714_163622' # with weight height, n_estimator = 20
# ---------------------------------------------------------------------------- #

# ------------------------------ filter app_age ------------------------------ #
# lr_out = 'hb_lr_output_20220726_154536' # keep app_age>=6; no weight height
# lr_out = 'hb_lr_output_20220726_173905' # keep app_age>=6; with weight height
# lr_out = 'hb_lr_output_20220726_154339' # keep app_age>=6; with weight height; filter weight by of all ages
# lr_out = 'hb_lr_output_20220727_135834' # keep app_age>=18; no weight height
# lr_out = 'hb_lr_output_20220727_135726' # keep app_age>=18; with weight height
# ---------------------------------------------------------------------------- #



# ------------------------ filter weight, height, bmi ------------------------ #
# lr_out = 'hb_lr_output_20220725_144238' # filter all app_ages. with weight height as features.
# lr_out = 'hb_lr_output_20220725_144055' # filter app_ages<=5 only. no weight height as features.
# ---------------------------------------------------------------------------- #


# --------------- adjustment in bmi for age<=1, by bmi=bmi_clip -------------- #
# lr_out = 'hb_lr_output_20220719_114540' # weight height; n_estimator=120; test data in train; adjustment in bmi for age <= 1. result not as good
# lr_out = 'hb_lr_output_20220719_114421' # no weight height; n_estimator=120; test data in train; adjustment in bmi for age <= 1. result bad
# lr_out = 'hb_lr_output_20220719_114223' # no weight height; n_estimator=120; no test data in train; adjustment in bmi for age <= 1
# ---------------------------------------------------------------------------- #


# ---------------- adjustment in bmi for age<=1, by bmi=median --------------- #
# lr_out = 'hb_lr_output_20220722_113524' # no weight height; n_estimator=120
# lr_out = 'hb_lr_output_20220722_113745' # with weight height; n_estimator=20
# ---------------------------------------------------------------------------- #


# lr_out = 'hb_lr_output_20220721_101325' # no weight height, use rv3. n_estimator=120. This version is for debugging purposes.

# use weight and height
# lr_out = 'hb_lr_output_20220705_104026' # with higher lower bound for max_depth, n_estimator. Use weight and height. overfit. result ok. low decline in prefer
# lr_out = 'hb_lr_output_20220706_122409' # use `weight` and `height` as well. No change in `n_estimator` and `max_depth`. result good. This variation thinks occupation risk is important

# no change in weight and height
# lr_out = 'hb_lr_output_20220706_122621' # with higher lower bound for n_estimator, and lower bound for max_depth is 4. result not good. high decline in prefer
# lr_out = 'hb_lr_output_20220706_122536' # with higher lower bound for n_estimator only. lower bound is 120. result good. This variation thinks occupation risk not important
# lr_out = 'hb_lr_output_20220707_135752' # with higher lower bound for n_estimator only. lower bound is 200. not as good

# lr_out = 'hb_lr_output_20220704_153632' # with higher lower bound for max_depth, n_estimator. lower bound for max_depth is 7. result not good. overfit.

# lr_out = 'hb_lr_output_20220622_153846' # adj_bmi = 3
# lr_out = 'hb_lr_output_20220622_154333' # adj_bmi = 2

# lr_out = 'hb_lr_output_20220630_120625' # like 20220622_154333. Just to make sure after updating the code, the models are unaffected. with dlr only. no use
# lr_out = 'hb_lr_output_20220630_112716' # like 20220622_154333. Just to make sure after updating the code, the models are unaffected

# lr_out = 'hb_lr_output_20220630_180326' # adj_bmi=1
# lr_out = 'hb_lr_output_20220701_060425' # no adj_bmi, no weight, height, bmi

pred_col_names = ['dlr', 'freq_sev', 'cla_reg']
# pred_col_names = ['dlr']
# ---------------------------------------------------------------------------- #

cohort_col = ['age_gp','sex_male']
bin_chosen_pred_col_name = f"bin_{pred_col_names[0]}"
chosen_pred_col_name = pred_col_names[0]

n_bins = 100 # if n_bins = 100, then do percentile

tables = {
    'lr_table': ('hb', 'hb_poc_premiums_rv3'),
    'lr_out': ('temp', lr_out),
    'res_out': ('temp', '_'.join(['f2_thresh', 'lr', re.search('\d+_\d+', lr_out)[0], timestamp ])),
    # 'res_out': ('temp', ('_'.join(['f2_thresh', 'lr', re.search('\d+_\d+', lr_out)[0], timestamp]) + ('_offset' if bmi_offset else ''))),
    'res_out_offset': ('temp', ('_'.join(['f2_thresh', 'lr', 'offset', re.search('\d+_\d+', lr_out)[0],  timestamp]) )),
    'split'   : ('temp', 'hb_poc_splits'),
    
    # 'res_out_no_ts':('temp', ('_'.join(['f2_thresh', 'lr', re.search('\d+_\d+', lr_out)[0]]) + ('_offset' if bmi_offset else ''))),
    # 'res_out': ('temp', '_'.join(['thresh',lr_out, timestamp])),
    # 'res_out': ('temp', '_'.join(['thresh',lr_out])),
}

print(tables)

output_path = Path('thresh_output')
output_path.mkdir(parents=True, exist_ok=True)

try:
    with open('_timestamp.txt', 'x') as f:
        f.write(f"{timestamp}\n")
        # f.write(f"\n{tables['res_out'][1]}")
        f.write('_'.join(['f2_thresh', 'lr', re.search('\d+_\d+', lr_out)[0]]))
except FileExistsError:
    print('file _timestamp.txt already exists!')

In [ ]:
'_'.join(['f2_thresh', 'lr', re.search('\d+_\d+', lr_out)[0]])

In [ ]:
tables['res_out_offset']
tables['res_out']

In [ ]:
# tables
# table = '_'.join([table, timestamp])

In [ ]:
# this is for auto-reloading the self-defined modules if they have any changes
%load_ext autoreload
%autoreload 2

In [ ]:
def combine_records(df):
    # return a new df
    df_original = df.sort_values(['contract_no','year_seq'])

    # specify which columns need to be aggregated, and the corresponding aggregate function
    col_to_agg = {
        'exposure':'sum',
        # 'tot_pay_amt': lambda g: g.sum(min_count=1),
        'tot_pay_amt': 'sum',
        'eff_annual_prem':'sum',
        }
    
    df_by_contracts_agg = (
        df_original
        .groupby('contract_no')
        .agg(col_to_agg)
        .assign(
            loss_ratio = lambda df: df['tot_pay_amt']/df['eff_annual_prem'],
        )
    )
    
    return (
        df_original
        # .drop_duplicates('contract_no', keep='last')
        .drop_duplicates('contract_no', keep='first')
        .drop(list(col_to_agg.keys()), axis=1, errors='ignore')
        .merge(df_by_contracts_agg, on='contract_no')
    )

# data loading 

In [ ]:
useful_cols = [
    'contract_no',
    'annual_prem',
    'adj_annual_prem',
    'exposure',
    'year_seq',
    'tot_pay_amt',
    'inflate_tot_pay_amt',
    'bmi',
    'diff_low',
    'diff_high',
    'bmi_clip',
    'first_pol_substandard',
    'risk_class',
    'ins_sex',
    'app_age',
    'clean_record'
    
]

age_gp_interval = list(range(0, 56, 5)) + [np.inf]

# cols needed from lr model dataset
cols_table = ', '.join(f'lr_table.{col}' for col in useful_cols)
# cols needed from lr output dataset
# cols_out = ', '.join(f'lr_out.{col}' for col in ['clean_record'] + pred_col_names)
cols_out = ', '.join(f'lr_out.{col}' for col in pred_col_names)

# df_lr_out = load_table(*tables['lr_out'], columns=['contract_no', 'clean_record'] + pred_col_names)
# # df_lr_out = load_table(*tables['lr_out'], columns=['contract_no', 'clean_record','substandard'] + pred_col_names)
# # df_info = load_table(*tables['lr_table'], columns=useful_cols, where='year_seq<=5')
# # split = load_table(*tables['split'], columns=['contract_no', 'train_1']) 

q = f"""
SELECT {cols_table}, s.train_1, {cols_out}
FROM 
    {'.'.join(tables['lr_table'])} lr_table 
	JOIN {'.'.join(tables['split'])} s USING (contract_no) 
	JOIN {'.'.join(tables['lr_out'])} lr_out USING (contract_no)
WHERE lr_table.year_seq <= 5
"""

with dsar.psql_con("WRITE") as con:
    df_info = pd.read_sql(q, con=con)

In [ ]:
df_info[['adj_annual_prem', 'exposure']]

# data preprocessing

In [ ]:
lr_out = (
    df_info
    .assign(
        eff_annual_prem=lambda df: df.adj_annual_prem * df.exposure,
        age_gp=lambda df: pd.cut(df['app_age'], age_gp_interval, right=True, include_lowest=True)
        )
    .pipe(combine_records)
    .query(
        "risk_class == 'decline' or train_1 == 1 or (train_1==0 and exposure == 5)" 
        if use_expo_less_than_thresh 
        else "risk_class == 'decline' or exposure == 5"
        )
    # .query("risk_class == 'decline' or exposure == 5")
    .replace({
            'ins_sex': {'M':1, 'F':0},
            'risk_class': {
                'exclusion':'substandard', 
                'exclusion_loading': 'substandard', 
                'loading':'substandard'
                },
            })
    .rename({
        'ins_sex':'sex_male', 
        'first_pol_substandard': 'substandard', 
        'xgb': 'dlr',
        'freq_severity': 'freq_sev',
        }, axis=1
        )
    # .merge(df_lr_out, on='contract_no', how='inner')
    # .merge(split, on='contract_no', how='inner')
    
)

if remap_model:
    lr_out = lr_out.rename(remap_model, axis=1)
    pred_col_names = list(remap_model.values())
    bin_chosen_pred_col_name = f"bin_{pred_col_names[0]}"
    chosen_pred_col_name = pred_col_names[0]

# this is needed because we don't want to include decline records when calculating aggregate lr 
lr_out.loc[lr_out.risk_class=='decline', ['tot_pay_amt','eff_annual_prem', 'loss_ratio']] = np.nan

lr_out_train = lr_out.query("train_1==1").copy()
lr_out_test = lr_out.query("train_1==0").copy()

In [ ]:
# pred_col_names = ['dlr', 'freq_sev', 'cla_reg']
# bin_chosen_pred_col_name = f"bin_{pred_col_names[0]}"
# chosen_pred_col_name = pred_col_names[0]

In [ ]:
# lr_out_test.exposure.mean()
# lr_out_train.exposure.mean()

In [ ]:
# df_info.query("contract_no =='80000098'")

In [ ]:
temp = lr_out_test.query("risk_class=='standard'")

find_loss_ratio(temp)


# temp.tot_pay_amt.sum()/((temp.annual_prem * temp.exposure).sum())

temp[['contract_no', 'annual_prem', 'adj_annual_prem', 'exposure', 'eff_annual_prem']]


# temp.annual_prem*temp.exposure

# temp.eff_annual_prem
# temp.adj_annual_prem * temp.exposure


## find the bin of each record, according to predicted loss ratio

In [ ]:
for pred_col_name in pred_col_names:
    bins = (
        lr_out_train
        .groupby(cohort_col)[pred_col_name]
        # wingho. will create 100 bins
        .apply(lambda s: np.quantile(s, q=np.arange(1/n_bins, 1.0, 1/n_bins))) 
        
        # jason. will create 101 bins
        # .apply(lambda s: np.percentile(s, q=np.linspace(1,100,100))) 
    )
    
    for portion in [lr_out_test, lr_out_train]:
        portion[f'bin_{pred_col_name}'] = (
            portion
            .groupby(cohort_col)[pred_col_name]
            # wingho
            # .apply(find_rank, quantiles=bins, n_bins=n_bins) 
            
            # wingho. same as above
            .apply(lambda s: pd.Series(np.digitize(s, bins[s.name])+1, index=s.index))
            
            # jason
            # .apply(lambda s: pd.Series(
            #     np.digitize(s, np.insert(bins[s.name], 0, -1), right=True), index=s.index)
            #     )
        )

# sanity check

In [ ]:
df_riskclass_size = lr_out_test.risk_class.value_counts_plus(index_name='risk_class')
df_riskclass_cohort_size = lr_out_test.groupby(cohort_col).risk_class.pipe(value_counts_plus).unstack(-1)

df_riskclass_size
df_riskclass_cohort_size

df_riskclass_size.to_excel(output_path / "df_riskclass_size.xlsx")
df_riskclass_cohort_size.to_excel(output_path / "df_riskclass_cohort_size.xlsx")

## check app_age distribution

In [ ]:
df_app_age_size = lr_out_test.app_age.value_counts_plus(cum=True, index_name='app_age').sort_index()

df_app_age_size

df_app_age_size.to_excel(output_path / "df_app_age_size.xlsx")

df_app_age_size['rate_cum'].plot(xticks=range(0,81,5), xlim=(0,82), xlabel='app_age', ylabel='cumulative rate', yticks=np.arange(0,1.1,0.1), title='cumulative distribution of application age')

savefig_plus(output_path/f"df_app_age_size_{timestamp}.png", dpi=150)

In [ ]:
# lr_out
lr_out.query("bmi.between(38.54,45)")
# [['contract_no','bmi', 'clean_record','app_age']].sort_values(ascending=False, by='bmi')

## check cohort size, claim count and rate

In [ ]:
# cohort_size_cnt = lr_out_test.groupby(cohort_col).size().unstack()
# cohort_size_rate = lr_out_test.groupby(cohort_col).size().unstack()/len(lr_out_test)
# df_cohort_size = pd.concat([cohort_size_cnt, cohort_size_rate], axis=1, keys=['count','rate'])

cohort_size_cnt = pd.crosstab(index=lr_out_test['age_gp'], columns=lr_out_test['sex_male'], margins=True)
cohort_size_rate = pd.crosstab(index=lr_out_test['age_gp'], columns=lr_out_test['sex_male'], margins=True, normalize=True)
df_cohort_size = pd.concat([cohort_size_cnt, cohort_size_rate], axis=1, keys=['record_count','record_count_rate'])

df_cohort_size
# df_cohort_size.to_excel(output_path/f"df_cohort_size.xlsx")

lr_out_test_has_claim = lr_out_test.query("loss_ratio>0")
cohort_size_claim_cnt = pd.crosstab(index=lr_out_test_has_claim['age_gp'], columns=lr_out_test_has_claim['sex_male'], margins=True)
cohort_size_claim_cnt_rate = pd.crosstab(index=lr_out_test_has_claim['age_gp'], columns=lr_out_test_has_claim['sex_male'], margins=True, normalize=True)
df_cohort_size_claim = pd.concat([cohort_size_claim_cnt, cohort_size_claim_cnt_rate], axis=1, keys=['claim_count','claim_count_rate'])
df_cohort_size_claim

df_cohort_size = pd.concat([df_cohort_size, df_cohort_size_claim], axis=1)
df_cohort_size
df_cohort_size.to_excel(output_path/f"df_cohort_size.xlsx")

# 10% data in prefer

# lr_prefer is obtained from 16x people

## check the `risk_class` and `substandard` distribution

In [ ]:
for portion_name, portion in zip(['whole', 'train', 'test'], [lr_out, lr_out_train, lr_out_test]):
    print(f"# ------------------------ for {portion_name} dataset ------------------------ #")
    portion.risk_class.value_counts(dropna=False)
    portion.substandard.value_counts(dropna=False)
    portion.loss_ratio.mean()
    portion.groupby('risk_class').loss_ratio.mean()
    print("# ------------------------------------ end ----------------------------------- #")

## check cohort size and unique predictions

In [ ]:
for portion_name, portion in zip(['train','test'],[lr_out_train, lr_out_test]):
    for pred in pred_col_names:
        print(f"n_unqiue output for {pred} on {portion_name} set: {portion[pred].nunique()}")
        
# print("cohort size:")
df_sizes = pd.concat(
    [
        lr_out_train.groupby(cohort_col).size().unstack(), 
        lr_out_test.groupby(cohort_col).size().unstack(),
    ], 
    axis=1, 
    keys=['train','test'],
    )

# print("unique values:")
df_uniques = pd.concat(
    [
        lr_out_train.groupby(cohort_col)[chosen_pred_col_name].nunique().unstack(), 
        lr_out_test.groupby(cohort_col)[chosen_pred_col_name].nunique().unstack(),
    ], 
    axis=1, 
    keys=['train','test'],
    )

pd.concat([df_sizes, df_uniques], axis=1, keys=['n_records', 'n_unique'])
# pd.concat([df_sizes, df_uniques], axis=1, keys=['n_records', 'n_unique']).to_excel(output_path/f"n_record_vs_n_unique_{timestamp}.xlsx")
pd.concat([df_sizes, df_uniques], axis=1, keys=['n_records', 'n_unique']).to_csv(output_path/f"n_record_vs_n_unique_{timestamp}.csv")


print(f"for {chosen_pred_col_name} model, the frequency distribution of the prediction values is:")
(
lr_out_test[chosen_pred_col_name]
.value_counts(normalize=True)
.reset_index()
.rename({'index':'predict', 'dlr':'prop'}, axis=1)
[:30]
)

# lr_out = 'hb_lr_output_20220704_153632'
# n_unqiue output for dlr on train set: 28868
# n_unqiue output for freq_sev on train set: 52517
# n_unqiue output for cla_reg on train set: 36844
# n_unqiue output for dlr on test set: 14796
# n_unqiue output for freq_sev on test set: 23607
# n_unqiue output for cla_reg on test set: 18355


# n_unqiue output for dlr on train set: 11649
# n_unqiue output for freq_sev on train set: 52516
# n_unqiue output for cla_reg on train set: 14041
# n_unqiue output for dlr on test set: 6773
# n_unqiue output for freq_sev on test set: 23607
# n_unqiue output for cla_reg on test set: 7633

## check the size of each bin

In [ ]:
res = []
for portion in [lr_out_train, lr_out_test]:
    for pred in pred_col_names:
        res.append(portion[f'bin_{pred}'].value_counts(normalize=True).sort_index())

res = pd.concat(res, axis=1)

res = (
    res
    .set_axis(
    # pd.MultiIndex.from_arrays([['train','train','train','test','test','test'], res.columns]), 
    pd.MultiIndex.from_arrays(
        [['train']*len(pred_col_names) + ['test']*len(pred_col_names), res.columns]
        ), 
    axis=1)
    .rename_axis(['bin'], axis=0)
)
res

sns.relplot(data=res.loc[:,'train'], kind='line').set(title='train set')
sns.relplot(data=res.loc[:,'test'], kind='line').set(title='test set')

# adj_bmi = 3:
# dlr
# 5556
# 52366
# 16742
# 3304
# 23566
# 9094

# for chosen model in report:
# 21888
# 52394
# 38276
# 11835
# 23577
# 19028

# 11.9, 16.9
# get enoguh bins such that prefer has >=8% (10)
# get enoguh bins such that prefer has >=12%
# perfer : (8,12)
# standard: (60,75)
# (3)

## check the records with lowest and highest predicted loss ratio by cohort

In [ ]:
k = 5
# cols = ['substandard','risk_class', 'clean_record', 'bmi', chosen_pred_col_name]
# cols = ['risk_class', 'clean_record', 'diff_low', 'diff_high', 'bmi_clip', 'bmi', chosen_pred_col_name]
cols = ['clean_record', 'diff_low', 'diff_high', 'bmi_clip', 'bmi', chosen_pred_col_name]


low_k = (
    lr_out_test
    .groupby(cohort_col)[cols]
    .apply(lambda g: g.nsmallest(n=k, columns=chosen_pred_col_name, keep='first').reset_index(drop=True).rename_axis('top_record', axis=0))
)

high_k = lr_out_test.groupby(cohort_col)[cols].apply(lambda g: g.nlargest(n=k, columns=chosen_pred_col_name, keep='first').reset_index(drop=True).rename_axis('top_record', axis=0))

# low_k.bmi.describe()
# high_k.bmi.describe()

top_records_full = pd.concat([low_k,high_k], keys=[f'low_{k}',f'high_{k}'], axis=1)

# top_records_full
top_records = top_records_full.loc[:, (slice(None), ['clean_record', 'bmi', chosen_pred_col_name])]


top_records_full.to_excel(output_path / f"top_records_full_{timestamp}.xlsx")
top_records.to_excel(output_path / f"top_records_{timestamp}.xlsx")


top_records_full.head(2)

## check relation between bmi and bin

In [ ]:
def bmi_plot_cohort(test_only=True, bmi_offset=False):
    portion_names = ['test'] if test_only else ['test','train']
    portions = [lr_out_test] if test_only else [lr_out_test, lr_out_train]
    # for portion_name, portion in zip(['test','train'], [lr_out_test, lr_out_train]):
    for portion_name, portion in zip(portion_names, portions):
        g = sns.relplot(data=portion, x='bmi', y=bin_chosen_pred_col_name, row='age_gp', col='sex_male', kind='scatter', style='clean_record', facet_kws={'sharex':False}, aspect=1.5)
        
        for ax in g.axes.flat:
            _ = ax.set_xlabel('bmi')
        suptitle = f'bmi vs bin by cohort for {chosen_pred_col_name} model on {portion_name} set (higher bin means higher risk)' 
        if bmi_offset: 
            suptitle += ' (with offset)'
        g.figure.suptitle(suptitle, y=1.004, fontsize=15)
        g.tight_layout()
        
        fig_name = "bmi_vs_bin_cohort_offset" if bmi_offset else "bmi_vs_bin_cohort"
        savefig_plus(output_path/f"{fig_name}_{portion_name}_{timestamp}.png", dpi=150)

bmi_plot_cohort()

In [ ]:
# g = sns.relplot(data=lr_out_test, x='bmi', y=bin_chosen_pred_col_name, row='age_gp', col='sex_male', kind='scatter', style='clean_record', facet_kws={'sharex':False}, aspect=1.5)

# # g = sns.relplot(data=lr_out_test, x='bmi', y=bin_chosen_pred_col_name, row='age_gp', col='sex_male', kind='scatter', facet_kws={'sharex':False}, aspect=1.5)
# for ax in g.axes.flat:
#     _ = ax.set_xlabel('bmi')
# g.figure.suptitle(f'bmi vs bin by cohort for {chosen_pred_col_name} model (higher bin means higher risk)', y=1.004, fontsize=15)
# g.tight_layout()

# savefig_plus(output_path/f"bmi_vs_bin_cohort_{timestamp}.png", dpi=150)

# # savefig_plus(output_path/f"bmi_vs_bin_cohort_{timestamp}.pdf")

# # g = sns.relplot(data=lr_out_test, x='bmi', y=bin_chosen_pred_col_name, hue='age_gp', kind='scatter',facet_kws={'sharex':False}, aspect=2)
# # savefig_plus(output_path/f"bmi_vs_bin_overall_{timestamp}.png")

In [ ]:
lr_out_test

In [ ]:
# age_gp_interval_adult = [0,17,25,30,35,40,45,50,55,np.inf]
def bmi_plot_child_adult(test_only=True, bmi_offset=False, invert_risk=False):
    portion_names = ['test'] if test_only else ['test','train']
    portions = [lr_out_test.copy()] if test_only else [lr_out_test.copy(), lr_out_train.copy()]
    for portion_name, portion in zip(portion_names, portions):
        age_gp_child = pd.cut(portion.app_age, list([0,5,10,17]), right=True, include_lowest=True)
        age_gp_adult = pd.cut(portion.app_age, [18,25,30,35,40,45,50,55,np.inf], right=True, include_lowest=True)
        fig, axes = plt.subplots(1,2,figsize=(20,9))
        
        if invert_risk: 
            portion[bin_chosen_pred_col_name] = 100 - portion[bin_chosen_pred_col_name]
            suptitle = f"bmi vs health score for {chosen_pred_col_name} model on {portion_name} set (higher health score means lower risk)"
        else:
            suptitle = f"bmi vs bin for {chosen_pred_col_name} model on {portion_name} set (higher bin means higher risk)"
            
        ax = sns.scatterplot(data=portion.query("app_age<18"), x='bmi', y=bin_chosen_pred_col_name, hue=age_gp_child, style='sex_male', ax=axes[0])
        ax.set(title='app_age<18', 
               ylabel='health score (higher is lower risk)' if invert_risk else 'bin (higher is higher risk)',
               )
        # ax.legend(bbox_to_anchor =(1.3, 0.4))
        
        ax = (sns
              .scatterplot(data=portion.query("app_age>=18"), x='bmi', y=bin_chosen_pred_col_name, hue=age_gp_adult, style='sex_male', ax=axes[1])
              .set(title='app_age>=18', 
                   ylabel='health score (higher is lower risk)' if invert_risk else 'bin (higher is higher risk)',
                )
        )
        # suptitle = f"bmi vs health score for original model on {portion_name} set"
        # suptitle = f"bmi vs health score for updated model on {portion_name} set"
        # suptitle = f"bmi vs bin for {chosen_pred_col_name} model on {portion_name} set (higher bin means higher risk)"
        # suptitle = f"bmi vs health score for {chosen_pred_col_name} model on {portion_name} set (higher health score means lower risk)"
        
        # if bmi_offset: 
        #     suptitle += ' (with offset)'
        fig.suptitle(suptitle, y=0.95, fontsize=15)

        # fig.tight_layout()
        fig_name = "bmi_vs_bin_by_app_age_offset" if bmi_offset else "bmi_vs_bin_by_app_age"
        savefig_plus(output_path/f"{fig_name}_{portion_name}_{timestamp}.png", dpi=150)
    
bmi_plot_child_adult()
# for portion_name, portion in zip(['test','train'], [lr_out_test, lr_out_train]):
#     age_gp_child = pd.cut(lr_out_test.app_age, list([0,5,10,17]), right=True, include_lowest=True)
#     age_gp_adult = pd.cut(lr_out_test.app_age, [18,25,30,35,40,45,50,55,np.inf], right=True, include_lowest=True)
#     fig, axes = plt.subplots(1,2,figsize=(20,9))
#     g = sns.scatterplot(data=lr_out_test.query("app_age<18"), x='bmi', y=bin_chosen_pred_col_name, hue=age_gp_child, style='sex_male', ax=axes[0]).set(title='app_age<18')
#     g = sns.scatterplot(data=lr_out_test.query("app_age>=18"), x='bmi', y=bin_chosen_pred_col_name, hue=age_gp_adult, style='sex_male', ax=axes[1]).set(title='app_age>=18')
#     fig.suptitle(f"bmi vs bin for {chosen_pred_col_name} model (higher bin means higher risk)", y=0.95, fontsize=15)

#     savefig_plus(output_path/f"bmi_vs_bin_by_app_age_{timestamp}.png", dpi=150)

In [ ]:
def bmi_plot_baby(test_only=True, bmi_offset=False):
    portion_names = ['test'] if test_only else ['test','train']
    portions = [lr_out_test] if test_only else [lr_out_test, lr_out_train]
    for portion_name, portion in zip(portion_names, portions):
        data = portion.query("app_age.between(0,7)")
        if not data.empty:
            g = sns.relplot(data=data, x='bmi', y=bin_chosen_pred_col_name, hue='sex_male', col='app_age', col_wrap = 2, facet_kws={'sharex':False}, aspect=1.3, style='clean_record')
            
            suptitle = f'bmi vs bin of babies for {chosen_pred_col_name} model on {portion_name} set (higher bin means higher risk)'
            if bmi_offset: 
                suptitle += ' (with offset)'
            
            g.figure.suptitle(suptitle, y=1.004, fontsize=15)
            g.tight_layout()
            
            fig_name = "bmi_vs_bin_babies_offset" if bmi_offset else "bmi_vs_bin_babies"
            savefig_plus(output_path/f"{fig_name}_{portion_name}_{timestamp}.png", dpi=150)

bmi_plot_baby()

In [ ]:
# lr_out_test.query("age_gp == @pd.Interval(10,15) and sex_male==0 and bmi>35").sort_values('bin_dlr').sort_values('contract_no')[['contract_no','bin_dlr']] # similarly large bmi, but differing bins. This is due to occupational risk
# .to_excel("strange_record.xlsx") 

# lr_out_test.query("age_gp == @pd.Interval(15, 20) and sex_male==0 and bmi.between(48,53)").sort_values("contract_no")[['contract_no','bin_dlr']]  # both with high bmi and clean record, but given different bin. It seems that's because the one with lower bin is taller and heavier.


# lr_out_test.query("age_gp == @pd.Interval(20, 25) and sex_male==1 and bmi.between(48,51)") # two records with similarly high bmi, but one of them is clean record, and so one of them given low bin

# lr_out_test.query("age_gp == @pd.Interval(25,30) and sex_male==1 and bmi.between(58,61)") # high bmi, but due to clean record, still assigned low bin. Also, this person has very high weight. It seems the model thinks high weight is good.

# lr_out_test.query("age_gp == @pd.Interval(30,35) and sex_male==1 and clean_record==0 and bmi.between(15,20)").sort_values('bin_dlr') # very low bmi, no clean record, assigned a low bin 


# 83355824, 86861688	
# 80807254
# 84405169	
# 85561023	

# lr_out_test.query("contract_no in ['85561023', '80807254','84405169']")

In [ ]:
# sanity check: how many unique bin edges 
# bins = np.quantile(lr_out_test[chosen_pred_col_name], q = np.arange(1/100, 1, 1/100))
# bins
# np.unique(bins).shape
# lr_out_test[chosen_pred_col_name].value_counts(normalize=True)

# pd.Series(np.digitize(lr_out_test[pred_col_name], bins=bins)).value_counts().sort_index()

# threshold tunning

## helper functions for threshold tunning

In [ ]:
def find_stats(df, class_name, total_size, as_cohort=False, model_name=None, pref_bin=None, std_bin=None, margin_bin=None):
    d = {}
    
    if as_cohort:
        d['class'] = class_name
        d['model_name'] = model_name
        d['pref_bin'] = pref_bin
        d['std_bin'] = std_bin
        d['margin_bin'] = margin_bin
        
    d['prop'] = len(df)/total_size
    d['n'] = len(df)
    
    d.update(
        df
        .risk_class
        .value_counts(normalize=False)
        .rename({
            'standard': 'std_n',
            'substandard': 'substd_n',
            'decline': 'decline_n'
            })
        # .sort_index()
        .to_dict()
    )
    # print(d)
    
    for cls in['std','substd','decline']:
        if f"{cls}_n" not in d:
            d[f"{cls}_n"] = 0
        d[f"{cls}_prop"] = d[f"{cls}_n"]/d['n']
    
    # print(d['decline_n'])
    
    d['nonstd_prop'] = (d['substd_n'] + d['decline_n'])/d['n']
    
    d['lr'] = df.tot_pay_amt.sum()/df.eff_annual_prem.sum() if len(df) > 0 and df.eff_annual_prem.sum() != 0 else 0
    
    d['tot_pay_amt'] = df.tot_pay_amt.sum()
    d['eff_annual_prem'] = df.eff_annual_prem.sum()
    
    # d['lr'] = df.tot_pay_amt.sum()/df.eff_annual_prem.sum()
    
    # for these two classes, also calculate their precision (i.e., agreement rate)
    if class_name == 'P_S':
        d['precision'] = df.risk_class.eq('standard').mean()
    elif class_name == 'M_N':
        # d['precision'] = df.risk_class.ne('standard').mean()
        d['precision'] = df.risk_class.isin(['substandard','decline']).mean()
    else:
        d['precision'] = np.nan
    
    return d if as_cohort else pd.Series(d, name=class_name)
    # return pd.Series(d) if as_cohort else pd.Series(d, name=class_name)
    
def find_stats_cohort(df, class_name, total_size, cohort_col, model_name=None, pref_bin=None, std_bin=None, margin_bin=None):
    # df contains records in a particular class
    # print(df)
    
    # each g is a cohort in a particular class (pref, std, M_N)
    return df.groupby(cohort_col).apply(lambda g: find_stats(g, class_name, total_size, as_cohort=True, model_name=model_name, pref_bin=pref_bin, std_bin=std_bin, margin_bin=margin_bin)).apply(pd.Series).reset_index()
    # return df.groupby(cohort_col).apply(lambda df: find_stats(df, class_name, total_size, as_cohort=True, model_name=model_name, pref_bin=pref_bin, std_bin=std_bin, margin_bin=margin_bin)).apply(pd.Series).reset_index()

def find_pct_diff_lr(s1:pd.Series, s2:pd.Series, index_name='lr'):
    # return (s1[index_name]-s2[index_name])/(s2[index_name])*100
    return (s1[index_name]-s2[index_name])/(s2[index_name])

def find_pct_diff_lr_plus(s1:pd.Series, s2:pd.Series):
    s1_trim = s1[s1.index.str.startswith('lr')]
    s2_trim = s2[s2.index.str.startswith('lr')]
    return ((s1_trim-s2_trim)/s2_trim).add_prefix(f"prop_diff_{s1_trim.name}_{s2_trim.name}_")

# find_stats_cohort(lr_out_test, "M_N", len(lr_out_test))
find_stats_cohort(lr_out_test, "pref", len(lr_out_test), cohort_col=cohort_col)

# find_stats(lr_out_test, "prefer", len(lr_out_test))
find_stats(lr_out_test, "M_N", len(lr_out_test))

# prop                1.00000
# n               26711.00000
# std_n           24008.00000
# substd_n         2073.00000
# decline_n         630.00000
# std_prop            0.89881
# substd_prop         0.07761
# decline_prop        0.02359
# nonstd_prop         0.10119
# lr                  0.59039
# precision           0.10119
# Name: M_N, dtype: float64

In [ ]:
# def find_bin(lr_out_test=None, bin_to_prop=None, bin_col_name=None, sizes=None, prev_bin=None, ):
def find_bin(s=None, sizes=None, prev_bin=None):
    # prev_bin: the ending bin of the previous risk_class.
    # if given, then find the bin number `res` (start counting from from prev_bin +1) such that the proportion of data from {prev_bin+1, prev_bin+2,..., res} = size
    # print(bin_to_prop)
    
    # ---------------------- directly operate on Series ---------------------- #
    # treat `index` as bin number
    bin_to_prop = s.value_counts(normalize=True).sort_index()
    
    # print(bin_to_prop)
    
    if prev_bin:
        # bin_to_prop = bin_to_prop.loc[prev_bin+1:]
        bin_to_prop = bin_to_prop[bin_to_prop.index>prev_bin]
    
    return [
        bin_to_prop
        # .index[bin_to_prop.cumsum().ge(size).argmax()] 
        .index[
            # find the first bin with cumulative proportion larger than size
            res[0] if (res:=np.where(bin_to_prop.cumsum().gt(size))[0]).size>0 
            # if such bin doesn't exist, then return the last bin
            else len(bin_to_prop)-1 # return the last element
            ]
        for size in sizes
    ]    


# ------------------------------- testing code ------------------------------- #
s = lr_out_test[bin_chosen_pred_col_name]
bin_to_prop = s.value_counts(normalize=True).sort_index()
# find_bin(bin_to_prop, sizes=[0.1, 0.12], prev_bin=None)
# find_bin(s, sizes=[0.81], prev_bin=1)
# find_bin(s, sizes=[0.2,0.5], prev_bin=1)
find_bin(s, sizes=[0.2,0.5])
# find_bin(s, sizes=[0.75], prev_bin=1)
# find_bin(bin_to_prop, sizes=[0.75], prev_bin=1)
# find_bin(lr_out_test, bin_chosen_pred_col_name, sizes=[0.1, 0.12], prev_bin=None)

# res = bin_to_prop.iloc[bin_to_prop.prop.cumsum().ge(size).argmax(), 0]
# print(res)
# return res

# bin_to_prop = bin_to_prop.reset_index().set_axis(['bin','prop'], axis=1)
# print(bin_to_prop.cumsum().gt(0.09).argmax())
# print(bin_prefer)
# temp = bin_to_prop.query("bin>@bin_prefer")
# bin_std = temp.iloc[temp.prop.cumsum().ge(size_dict['std']).argmax(), 0]
# print(bin_std)

# temp = bin_to_prop.query("bin>@bin_std")
# bin_margin = temp.iloc[temp.prop.cumsum().ge(size_dict['margin']).argmax(), 0]
# print(bin_margin)

# size_dict = {'prefer':0.1, 'std':0.8, 'margin':0.02}
# find_bin(lr_out_test, 'bin_dlr', size=0.8, prev_bin=18)
# find_bin(lr_out_test, 'bin_dlr', sizes=[0.1, 0.11], prev_bin=None)

bin_to_prop.cumsum().loc[51]
# len(s[s<=18])/len(s)

## function to do threshold tuning
- index i, j, k:
    - i: the ending bin of prefer class (inclusive)
        - prefer class has bins between 1 and i
    - j: the ending bin of std class (inclusive)
        - std class has bins between i+1 and j
    - k: the ending bin of margin class (inclusive)
        - margin class has bins between j+1 and k
    - k+1: the starting bin of nonstd class (inclusive)
        - nonstd class has bins between k+1 and 100

- usual range of size for each risk class
    - prefer: 8-12
    - standard: 73-82
    - margin: 1-4
    - nonstd: 7-13

In [ ]:
def evaluate_lr_threshold(lr_out_test, n_bins, bin_col_name, model_name, cohort_col):
    total_size = len(lr_out_test)
    lr_out_test = lr_out_test.sort_values(bin_col_name) # sort according to the given model prediction
    
    # find the starting index of each bin. Note: the bins have already been sorted in ascending order
    bin_start = pd.Series(
        lr_out_test[bin_col_name]
        .searchsorted(range(1,n_bins+1)), index=range(1,n_bins+1)
        )
    
    # print(bin_start)
    
    # this will store the results of all bin combinations
    res = []
    
    res_cohort = []

    # find the starting bin and ending bin for prefer class
    range_start, range_end = find_bin(
        s=lr_out_test[bin_col_name], 
        sizes=[0.07, 0.13]
        # sizes=[0.08, 0.12]
    )
    # print('i')
    # print(range_start, range_end+1)
    
    # for i in range(range_start, range_end+1): 
    for i in range(range_start, range_end): 
        data_pref = lr_out_test[:bin_start[i+1]]
        stats_pref = find_stats(df=data_pref, class_name='pref', total_size=total_size)
        
        # find the starting bin and ending bin for std class
        range_start, range_end = find_bin(
            s=lr_out_test[bin_col_name], 
            sizes=[0.73, 0.83], 
            # sizes=[0.82, 0.83],  # for testing only
            prev_bin=i
            )
        # print('j')
        # print(range_start, range_end+1)
        
        # for j in range(range_start, range_end+1):
        for j in range(range_start, range_end):
            # print(bin_start[i+1], bin_start[j+1])
            data_std = lr_out_test[bin_start[i+1]: bin_start[j+1]]
            
            stats_std = find_stats(df=data_std, class_name='std', total_size=total_size)
            stats_P_S = find_stats(
                df=pd.concat([data_pref, data_std], axis=0), 
                class_name='P_S', 
                total_size=total_size
                )
            
            # find the starting bin and ending bin for margin class
            range_start, range_end = find_bin(
                s=lr_out_test[bin_col_name], 
                sizes=[0.01, 0.03], 
                prev_bin=j
                )
            # for k in range(j+1, n_bins):
            # print('k')
            # print(range_start, range_end+1)
            
            # for k in range(range_start, range_end+1):
            for k in range(range_start, range_end):
                # if k is already the last bin, then doing `bin_start[k+1]` will result in error...
                data_margin = lr_out_test[bin_start[j+1]: bin_start[k+1]]
                data_nonstd = lr_out_test[bin_start[k+1]:]
                
                stats_margin = find_stats(df=data_margin, class_name='margin', total_size=total_size)
                
                stats_nonstd = find_stats(df=data_nonstd, class_name='nonstd', total_size=total_size)
                
                
                data_M_N = pd.concat([data_margin, data_nonstd], axis=0)
                stats_M_N = find_stats(
                    df=data_M_N, 
                    class_name='M_N', 
                    total_size=total_size
                    )
                
                for df, class_name in zip([data_pref, data_std, data_M_N], ['pref', 'std', 'M_N']):
                    res_cohort.append(find_stats_cohort(df=df, class_name=class_name, total_size=len(df), cohort_col=cohort_col, model_name=model_name, pref_bin=i, std_bin=j, margin_bin=k))
                # res_cohort.append(find_stats_cohort(df=data_std, class_name='std', total_size=total_size, model_name=model_name, pref_bin=i, std_bin=j, margin_bin=k))
                # res_cohort.append(find_stats_cohort(df=data_M_N, class_name='M_N', total_size=total_size, model_name=model_name, pref_bin=i, std_bin=j, margin_bin=k))
                
                # print(i,j,k)
                misc_info = pd.Series({
                    'model_name': model_name,
                    'pref_bin': i,
                    'std_bin': j,
                    'margin_bin': k,
                    # 'prop_diff_pref_std_lr': find_pct_diff_lr(stats_pref, stats_std),
                    # 'prop_diff_std_M_N_lr': find_pct_diff_lr(stats_std, stats_M_N),
                    # 'prop_diff_std_margin': find_pct_diff_lr(stats_std, stats_margin),
                    # 'prop_diff_margin_nonstd': find_pct_diff_lr(stats_margin, stats_nonstd),
                })
                
                misc_info = pd.concat([
                    misc_info, 
                    find_pct_diff_lr_plus(stats_pref, stats_std),
                    find_pct_diff_lr_plus(stats_std, stats_M_N),
                    ], axis=0)
                
                # print(misc_info)
                
                # each appended dataframe has one row only
                res.append(pd.concat({
                    '': misc_info, 
                    'pref': stats_pref, 
                    'std': stats_std, 
                    'M_N': stats_M_N, 
                    'margin': stats_margin, 
                    'nonstd': stats_nonstd, 
                    'P_S': stats_P_S,
                    },
                    axis=0
                ).to_frame().T
                )
                
    # print(len(res))
    return pd.concat(res, axis=0, ignore_index=True), pd.concat(res_cohort, axis=0, ignore_index=True)

# a, b = evaluate_lr_threshold(
#         lr_out_test, n_bins=n_bins, bin_col_name=f"bin_dlr", model_name=pred_col_name
#     )
# a
# b[:24*3+1]

res_indiv, res_cohort = [], []
for pred_col_name in pred_col_names:
# for pred_col_name in pred_col_names[:1]:
    df_indiv, df_cohort = evaluate_lr_threshold(
        lr_out_test, n_bins=n_bins, bin_col_name=f"bin_{pred_col_name}", model_name=pred_col_name, cohort_col=cohort_col
    )
    res_indiv.append(df_indiv)
    res_cohort.append(df_cohort)
df_res = pd.concat(res_indiv, axis=0)
df_res

df_res_cohort = pd.concat(res_cohort, axis=0)
df_res_cohort


## create new df to put important columns to front

In [ ]:
def reorder_columns(df_res):
    # ------------------------- flatten the column index ------------------------- #
    df_res_flat = (
        df_res
        .set_axis(['_'.join(s for s in col if s).strip() for col in df_res.columns.values], axis=1)
        # .sort_index(axis=1)
        # .sort_index(axis=1, key=lambda s: s.str.extract(r'_([^_]*)$')) # sort by the last suffix 
        # .sort_index(axis=1, key=lambda s: res if (res:= s.str.extract(r'_([^_]*)$')) is not None else 'a') # sort by the last suffix 
    )

    # --------------- convert columns of int type back to int type --------------- #
    # [col for col in df_res_flat.columns if re.search(r'_n$|_bin$', col)]
    cols_int = df_res_flat.columns[df_res_flat.columns.str.contains(pat=r'_n$|_bin$')] # those with suffix `_n` or `_bin`
    df_res_flat[cols_int] = df_res_flat[cols_int].astype(int)

    # ------------- put these columns to the front for easy reference ------------- #
    cols_to_front = [
        'model_name',
        'pref_bin',
        'std_bin',
        'margin_bin',
        'pref_prop',
        'std_prop',
        'margin_prop',
        'M_N_prop',
        'prop_diff_pref_std_lr',
        'prop_diff_std_M_N_lr',
        'pref_decline_prop',
        'std_decline_prop',
        'M_N_decline_prop',
        # 'pref_substd_prop',
        # 'std_substd_prop',
        # 'M_N_substd_prop',
        'P_S_precision',
        'M_N_precision',
        # 'nonstd_prop',
    ]

    # df_res_flat = df_res_flat[cols_to_front + [ col for col in df_res_flat.columns if col not in cols_to_front ]]
    # return df_res_flat
    return df_res_flat[cols_to_front + [ col for col in df_res_flat.columns if col not in cols_to_front ]]

df_res_flat = reorder_columns(df_res)
df_res_flat.head(2)
# list(df_res_flat.columns)
# df_res_flat.head(2)

## create new df for documenting on sheet

In [ ]:
def make_df_excel(df_res_flat, schema, table):
    cols_excel = [
        'model_name',
        'pref_bin',
        'std_bin',
        'margin_bin',
        
        'P_S_lr',
        'prop_diff_P_S_HS', # placeholder
        'P_S_prop',
        'P_S_precision',
        
        'M_N_lr',
        'prop_diff_M_N_HS', # placeholder
        'M_N_prop',
        'M_N_precision',
        
        'prop_diff_pref_std_lr',
        'prop_diff_std_M_N_lr',
        
        'pref_lr',
        'std_lr',
        'margin_lr',
        'nonstd_lr',
        
        'prop_diff_pref_HS', # placeholder
        'prop_diff_std_HS', # placeholder
        'prop_diff_margin_HS', # placeholder
        'prop_diff_nonstd_HS', # placeholder

        'pref_prop',
        'std_prop',
        'margin_prop',
        'nonstd_prop',
        
        'pref_decline_prop',
        'std_decline_prop',
        'M_N_decline_prop',
        
        # 'pref_substd_prop',
        # 'std_substd_prop',
        # 'M_N_substd_prop',
        
        # 'pref_decline_prop',
        # 'std_decline_prop',
        # 'M_N_decline_prop',
        
        'pref_nonstd_prop',
        'std_nonstd_prop',
        'M_N_nonstd_prop',
        
        'table',
        'combo',
        'sql',
    ]

    # schema, table = tables['res_out']
    df_res_excel = (
        df_res_flat
        .assign(
            prop_diff_P_S_HS = 0,
            prop_diff_M_N_HS = 0,
            prop_diff_pref_HS = 0,
            prop_diff_std_HS = 0,
            prop_diff_margin_HS = 0,
            prop_diff_nonstd_HS = 0,
            table=f"{schema}.{table}",
            combo='',
            sql = df_res_flat.apply(
                lambda row: f"SELECT * FROM {schema}.{table} WHERE pref_bin = {row.pref_bin} AND std_bin = {row.std_bin} AND margin_bin = {row.margin_bin} AND model_name = '{row.model_name}'", axis=1
                )
        )
        [cols_excel]
    )
    return df_res_excel 

df_res_excel = pd.concat([
    make_df_excel(df_res_flat, schema=tables['res_out'][0], table=tables['res_out'][1]), 
    df_res_flat.filter(regex=r'prop_diff_pref_std_lr_|prop_diff_std_M_N_lr_', axis=1).sort_index(axis=1)
    ], axis=1)

df_res_excel.head(2)

# df_res_excel = pd.concat([
#     df_res_excel, df_res_flat.filter(regex=r'prop_diff_pref_std_lr_|prop_diff_std_M_N_lr_', axis=1).sort_index(axis=1)
#     ], axis=1)
# # df_res_excel

# export data to db and local machine

In [ ]:
def export_results(df_res_flat, df_res_excel, df_res_cohort, schema, table):
    # schema, table = tables['res_out']
    # _, table_no_ts = tables['res_out_no_ts']
    # print(table)

    # table = '_'.join([table, timestamp])

    print(f"{schema = }")
    print(f"{table = }")
    print(f"original df output name: df_res_flat_{table}.csv")
    print(f"excel format name: df_res_excel_{table}.xlsx")
    print(f"table name in db: {table}")
    print(f"table name by cohort in db: {'_'.join([table,'cohort'])}")

    # ------------------------------ output results ------------------------------ #
    df_res_flat.to_excel(output_path/f"df_res_flat_{table}.xlsx")
    df_res_excel.to_excel(output_path/f"df_res_excel_{table}.xlsx")
    export_to_db(df_res_flat, schema=schema, table=table, if_exists='replace')
    export_to_db(df_res_cohort.astype({'age_gp':'str'}), schema=schema, table='_'.join([table,"cohort"]), if_exists='replace')
    # export_to_db(df_res_flat, schema=schema, table=table_no_ts, if_exists='replace')
    # export_to_db(df_res_cohort.astype({'age_gp':'str'}), schema=schema, table='_'.join([table_no_ts,"cohort"]), if_exists='replace')

    # df_res_excel.to_csv(output_path/'_'.join([tables['lr_out'][1], timestamp, "df_res_excel.csv"]))
    # tables
    
export_results(df_res_flat, df_res_excel, df_res_cohort, schema=tables['res_out'][0], table=tables['res_out'][1])

# bmi offset

In [ ]:
if bmi_offset:
    lr_out_test_offset = lr_out_test.copy()

In [ ]:
# adjust bmi by cohort
# global_record_count = 0
# def add_offset_bmi(g):
#     global global_record_count
#     low_limit = 0.003
#     high_limit = 0.997
#     base = 40
#     bmi_low, bmi_high = g.bmi.quantile(low_limit), g.bmi.quantile(high_limit)
#     print(f"for cohort {g.name}: ")
#     print(bmi_low, bmi_high)
#     print("number of affected records: ")
#     global_record_count += g.query(f"~bmi.between({bmi_low}, {bmi_high})").shape[0]
#     print(g.query(f"~bmi.between({bmi_low}, {bmi_high})").shape[0])
    
#     for pred_col_name in pred_col_names:
#         g[pred_col_name] = np.where(
#             g.bmi.between(bmi_low, bmi_high),
#             g[pred_col_name],
#             np.clip(g[pred_col_name] + base, a_min=1, a_max=100)
#             # np.clip(g.bin_dlr + base + , a_min=0, a_max=100)
#         )
#     return g

# # lr_out_test = lr_out_test.groupby(cohort_col).apply(add_offset_bmi)
# lr_out_test.groupby(cohort_col).apply(add_offset_bmi)
# global_record_count

# # lr_out_test.groupby(cohort_col).apply(play)


In [ ]:
# low_limit = 0.002
# high_limit = 0.998
# bmi_low, bmi_high = lr_out_test_offset.bmi.quantile(low_limit), lr_out_test_offset.bmi.quantile(high_limit)
# (lr_out_test_offset.bmi.between(bmi_low, bmi_high)) | (lr_out_test_offset[f"bin_{pred_col_name}"] > 50)

In [ ]:
# M1: add a constant to all record with extreme bmi
if bmi_offset:
    low_limit = 0.002
    high_limit = 0.998
    base = 50
    bmi_low, bmi_high = lr_out_test_offset.bmi.quantile(low_limit), lr_out_test_offset.bmi.quantile(high_limit)
    bmi_low, bmi_high
    for pred_col_name in pred_col_names:
        # print(pred_col_name)
        lr_out_test[f"bin_{pred_col_name}"] = np.where(
            lr_out_test_offset.bmi.between(bmi_low, bmi_high),
            # (lr_out_test_offset.bmi.between(bmi_low, bmi_high)) | (lr_out_test_offset[f"bin_{pred_col_name}"] > 50),
            lr_out_test_offset[f"bin_{pred_col_name}"],
            np.clip(lr_out_test_offset[f"bin_{pred_col_name}"] + base, a_min=1, a_max=100)
            # np.clip(lr_out_test_offset.bin_dlr + base + , a_min=0, a_max=100)
            )

    # show the records that have received offset
    lr_out_test.query(f"~bmi.between({bmi_low}, {bmi_high})")
    lr_out_test.query(f"~bmi.between({bmi_low}, {bmi_high})").shape

In [ ]:
# M2: add a constant and a linear penalty to records with extreme bmi
# if bmi_offset:
#     base = 20
#     multiplier = 3
#     bmi_low, bmi_high = lr_out_test_offset.bmi.quantile(low_limit), lr_out_test_offset.bmi.quantile(high_limit)

#     lr_out_test['bin_dlr'] = (
#         lr_out_test_offset.bin_dlr + 
#         lr_out_test_offset.bmi.apply(
#             lambda v: base + ceil(v-bmi_high)*multiplier if v>=bmi_high
#             else 0 if bmi_low< v< bmi_high
#             else base + ceil(bmi_low - v)*multiplier 
#             )
#     ).clip(upper=100)

#     # lr_out_test_offset.query(f"~bmi.between({bmi_low}, {bmi_high})")
#     lr_out_test.query(f"~bmi.between({bmi_low}, {bmi_high})")
#     # # lr_out_test.query(f"~bmi.between({bmi_low}, {bmi_high})").shape

## check relation between bmi and bin after bmi offset

In [ ]:
if bmi_offset:
    bmi_plot_cohort(bmi_offset=True)
    bmi_plot_child_adult(bmi_offset=True)
    bmi_plot_baby(bmi_offset=True)

# tuning after bmi offset

In [ ]:
if bmi_offset:
    # ---------------------------- get tunning results --------------------------- #
    res_indiv, res_cohort = [], []
    for pred_col_name in pred_col_names:
    # for pred_col_name in pred_col_names[:1]:
        df_indiv, df_cohort = evaluate_lr_threshold(
            lr_out_test, n_bins=n_bins, bin_col_name=f"bin_{pred_col_name}", model_name=pred_col_name, cohort_col=cohort_col
        )
        res_indiv.append(df_indiv)
        res_cohort.append(df_cohort)
    df_res = pd.concat(res_indiv, axis=0)
    df_res

    df_res_cohort = pd.concat(res_cohort, axis=0)
    df_res_cohort
    
    # ------------------------------ reorder columns ----------------------------- #
    df_res_flat = reorder_columns(df_res)
    df_res_flat.head(2)
    
    df_res_excel = pd.concat([
        make_df_excel(df_res_flat, schema=tables['res_out_offset'][0], table=tables['res_out_offset'][1]), 
        df_res_flat.filter(regex=r'prop_diff_pref_std_lr_|prop_diff_std_M_N_lr_', axis=1).sort_index(axis=1)
        ], axis=1)

    df_res_excel.head(2)
    
    # ------------------------------ export results ------------------------------ #
    export_results(df_res_flat, df_res_excel, df_res_cohort, schema=tables['res_out_offset'][0], table=tables['res_out_offset'][1])


# adhoc code

## digression: experiment to illustrate data with duplicates

In [ ]:
# bins = np.quantile([1,2], q=np.arange(1/100, 1, 1/100))

# data = [1, 1.0001, 1.9, 2]
# data = [1, 1.01, 1.01, 20, 30,  99.99, 99.9999, 100]
# data = [1, 1.01, 1.001, 20, 30,31,99,12, 100]
# data = [1,1,1,1,1,1.01, 1.02, 2,3,4]


# Q: why the bin edges aren't unique when we have duplicate values? Eg., why we have some bin edges being the same when we have many values of `1`?
# A: because about 10% of the data is 1, and so `1` will occupies 10 bins, because each bin should contain about 1% of the data
# data = [1]*10 + list(range(1,101,1))
# print("data: ")
# print(data)

# # data_unique = pd.Series(data).drop_duplicates()
# # bins = np.quantile(data_unique, q=np.arange(1/100, 1, 1/100))
# bins = np.quantile(data, q=np.arange(1/100, 1, 1/100))

# print("bin edges:")
# bins

# print("")
# np.digitize(data, bins=bins)


In [ ]:
# g = sns.catplot(data=df_prop_diff_cohort.query(f"pref_bin=={pref_bin} and std_bin=={std_bin} and margin_bin=={margin_bin}"), x='prop_diff', y='age_gp', ci=None, row='sex_male', col='comb', kind='bar', sharex=False).set(xlabel='proportion difference')
# g.refline(x=0, color='r', lw=3, label='diff = 0')
# g.figure.suptitle('proportion difference between predicted risk classes', fontsize=15, y=1.01)
# g.tight_layout()
# g.add_legend()

## use pointsize to distinguish record count size

In [ ]:
# data = df_res_cohort.query("pref_bin==12 and std_bin==91 and margin_bin==93").astype({'age_gp':'str'})
# g = sns.relplot(data=data, y='lr', x=data['age_gp'].astype('str'), kind='line',  hue='class', col='sex_male', style='class', ci=None, aspect=1.3, markers=True, markersize=7, lw=2).set(ylabel='loss ratio')
# g.figure.suptitle('loss ratio of each predicted risk class', fontsize=15)
# g.tight_layout()
# g.set_xticklabels(rotation=30, ha="right")


# for col, ax in g.axes_dict.items():
#     ax2 = ax.twinx()
#     ax2.tick_params(axis='y', colors='b') 
#     ax2.grid(visible=False)
#     # ax2.set_ylim(0,100)
#     # print(data.query("sex_male == @col"))
#     # sns.lineplot(data=data.query("sex_male == @col"), x='age_gp', y='n', ax=ax2, ci=None, marker=True, markers='*', markersize=30, linestyle='--', color='b')
#     sns.pointplot(data=data.query("sex_male == @col"), x='age_gp', y='n', hue='class', ax=ax2, ci=None, markers='o', linestyles='', scale=0.8)
#     ax2.set_ylabel('record count')
# g.tight_layout()


# g = sns.relplot(data=data, y='lr', x=data['age_gp'].astype('str'), kind='scatter',  hue='class', col='sex_male', aspect=1.3, size='n', sizes=(10,100)).set(ylabel='loss ratio', )

In [ ]:
# def find_stats(df, class_name, total_size):
#     d = {}
#     d['prop'] = len(df)/total_size
#     d['n'] = len(df)
#     d.update(
#         df
#         .risk_class
#         .value_counts(normalize=False)
#         .rename({
#             'standard': 'std_n',
#             'substandard': 'substd_n',
#             'decline': 'decline_n'
#             })
#         .to_dict()
#     )
    
#     for cls in['std','substd','decline']:
#         d[f"{cls}_prop"] = d[f"{cls}_n"]/d['n']
    
#     d['nonstd_prop'] = (d['substd_n'] + d['decline_n'])/d['n']
    
    
#     d['lr'] = df.tot_pay_amt.sum()/df.eff_annual_prem.sum() if len(df) > 0 and df.eff_annual_prem.sum() != 0 else 0
#     # d['lr'] = df.tot_pay_amt.sum()/df.eff_annual_prem.sum()
    
    
#     # for these two classes, also calculate their precision (i.e., agreement rate)
#     if class_name == 'P_S':
#         d['precision'] = df.risk_class.eq('standard').mean()
#     elif class_name == 'M_N':
#         # d['precision'] = df.risk_class.ne('standard').mean()
#         d['precision'] = df.risk_class.isin(['substandard','decline']).mean()
#     else:
#         d['precision'] = np.nan
        
#     return pd.Series(d, name=class_name)

# def find_pct_diff_lr(s1:pd.Series, s2:pd.Series, index_name='lr'):
#     # return (s1[index_name]-s2[index_name])/(s2[index_name])*100
#     return (s1[index_name]-s2[index_name])/(s2[index_name])

# # find_stats(lr_out_test, "prefer", len(lr_out_test))
# find_stats(lr_out_test, "M_N", len(lr_out_test))


# # prop                1.00000
# # n               26711.00000
# # std_n           24008.00000
# # substd_n         2073.00000
# # decline_n         630.00000
# # std_prop            0.89881
# # substd_prop         0.07761
# # decline_prop        0.02359
# # nonstd_prop         0.10119
# # lr                  0.59039
# # precision           0.10119
# # Name: M_N, dtype: float64

In [ ]:
# def find_stats(df, class_name, total_size):
#     d = {}
#     d['prop'] = len(df)/total_size
#     d['n'] = len(df)
#     d.update(
#         df
#         .risk_class
#         .value_counts(normalize=False)
#         .rename({
#             'standard': 'std_n',
#             'substandard': 'substd_n',
#             'decline': 'decline_n'
#             })
#         .to_dict()
#     )
    
#     for cls in['std','substd','decline']:
#         d[f"{cls}_prop"] = d[f"{cls}_n"]/d['n']
    
#     d['nonstd_prop'] = (d['substd_n'] + d['decline_n'])/d['n']
    
    
#     d['lr'] = df.tot_pay_amt.sum()/df.eff_annual_prem.sum() if len(df) > 0 else 0
#     # d['lr'] = df.tot_pay_amt.sum()/df.eff_annual_prem.sum()
    
#     # for these two classes, also calculate their precision (i.e., agreement rate)
#     if class_name == 'P_S':
#         d['precision'] = df.risk_class.eq('standard').mean()
#     elif class_name == 'M_N':
#         # d['precision'] = df.risk_class.ne('standard').mean()
#         d['precision'] = df.risk_class.isin(['substandard','decline']).mean()
    
    
#     # lr_cohort = df.groupby(cohort_col).apply(lambda gp: gp.tot_pay_amt.sum()/gp.eff_annual_prem.sum() if len(gp) > 0 else 0)
    
#     lr_cohort = (
#         df
#         .groupby(cohort_col)
#         .apply(lambda gp: gp.tot_pay_amt.sum()/gp.eff_annual_prem.sum() if len(gp) > 0 and gp.eff_annual_prem.sum() != 0 else 0)
#         .reset_index()
#     )
    
#     lr_cohort = (
#         lr_cohort
#         .set_index('lr_' + lr_cohort['age_gp'].astype('str') + '_' + lr_cohort['sex_male'].replace({0:'M', 1:'F'}))
#         .drop(['age_gp','sex_male'], axis=1)
#         .squeeze()
#     )
    
#     prop_cohort = df.groupby(cohort_col).size().div(len(df)).reset_index()
    
#     prop_cohort = (
#     prop_cohort
#     .set_index('prop_' + prop_cohort['age_gp'].astype('str') + '_' + prop_cohort['sex_male'].replace({0:'M', 1:'F'}))
#     .drop(['age_gp','sex_male'], axis=1)
#     .squeeze()
# )
    
#     decline_prop_cohort = df.groupby(cohort_col).apply(lambda g: len(g.query("risk_class=='decline'"))/len(g)).reset_index()
    
#     decline_prop_cohort = (
#     decline_prop_cohort
#     .set_index('decline_prop_' + decline_prop_cohort['age_gp'].astype('str') + '_' + decline_prop_cohort['sex_male'].replace({0:'M', 1:'F'}))
#     .drop(['age_gp','sex_male'], axis=1)
#     .squeeze()
# )
    
#     # return pd.Series(d, name=class_name), lr_cohort
#     return pd.concat([pd.Series(d), lr_cohort, prop_cohort, decline_prop_cohort], axis=0).rename(class_name)

# def find_pct_diff_lr(s1:pd.Series, s2:pd.Series, index_name='lr'):
#     # return (s1[index_name]-s2[index_name])/(s2[index_name])*100
#     return (s1[index_name]-s2[index_name])/(s2[index_name])

# def find_pct_diff_lr_plus(s1:pd.Series, s2:pd.Series):
#     s1_trim = s1[s1.index.str.startswith('lr')]
#     s2_trim = s2[s2.index.str.startswith('lr')]
#     return ((s1_trim-s2_trim)/s2_trim).add_prefix(f"prop_diff_{s1_trim.name}_{s2_trim.name}_")

# # find_stats(lr_out_test, "prefer", len(lr_out_test))
# find_stats(lr_out_test, "M_N", len(lr_out_test))
# # find_stats(lr_out_train, "prefer", len(lr_out_train))

# # prop                1.00000
# # n               26711.00000
# # std_n           24008.00000
# # substd_n         2073.00000
# # decline_n         630.00000
# # std_prop            0.89881
# # substd_prop         0.07761
# # decline_prop        0.02359
# # nonstd_prop         0.10119
# # lr                  0.59039
# # precision           0.10119
# # Name: M_N, dtype: float64

In [ ]:
# lr_out.groupby(cohort_col).risk_class.value_counts(normalize=True)
# lr_out.groupby(cohort_col).apply(lambda g: len(g.query("risk_class=='decline'"))/len(g))

In [ ]:
# s1 = find_stats(lr_out_test, "M_N", len(lr_out_test))
# s2 = find_stats(lr_out_train, "prefer", len(lr_out_train))
# s1
# s2
# # ((s1-s2)/s2).add_prefix(f"prop_diff_{s1.name}_{s2.name}_")

# find_pct_diff_lr_plus(s1,s2)

In [ ]:
# s1 = find_stats(lr_out_test, "M_N", len(lr_out_test))
# s1[s1.index.str.startswith('lr')]

In [ ]:
# lr_gp = lr_out.groupby(cohort_col).apply(lambda gp: gp.tot_pay_amt.sum()/gp.eff_annual_prem.sum() if len(gp) > 0 else 0).reset_index()
# lr_gp = lr_gp.set_index('lr_' + lr_gp['age_gp'].astype('str') + '_' + lr_gp['sex_male'].replace({0:'M', 1:'F'})).drop(['age_gp','sex_male'], axis=1).squeeze()
# lr_gp

# pd.concat([lr_gp,lr_gp], axis=0)

# lr_gp = lr_out.groupby(cohort_col).apply(lambda gp: gp.tot_pay_amt.sum()/gp.eff_annual_prem.sum() if len(gp) > 0 else 0)
# lr_gp.index
# lr_gp.index.map('_'.join)
# lr_gp.index.map(lambda s: str(s))

# lr_gp.index.astype(str).agg('|'.join, axis=1)

In [ ]:
# df_res.columns
# df_res.loc[:, 'misc']
# df_res[:1]
# df_res.loc[:, (slice(None), ['prop','pref_bin','std_bin','margin_bin'])]
# df_res 
# df_res.iloc[99:100]


# for col in df_res.columns.values:
#     '_'.join(s for s in col if s).strip()


In [ ]:
# useful_cols = [
# 'contract_no',
# 'annual_prem',
# 'adj_annual_prem',
# 'exposure',
# 'year_seq',
# 'tot_pay_amt',
# 'inflate_tot_pay_amt']

# columns = ', '.join(useful_cols) if useful_cols else '*'
# with dsar.psql_con("WRITE") as con:
#     df = pd.read_sql(sql=f"""select {columns} from {lr_model_table}
#                             where risk_class!='decline' and year_seq<=5 """, con = con)

# # df = df[df.year_seq<=5]
# df_lr_mapping = combine_records(df, use_adj_premium=use_adj_premium, use_inflate=use_inflate)
# # df_lr_mapping = df_lr_mapping[df_lr_mapping.exposure==5]

In [ ]:
# schema, table = tables['lr_out']
# with dsar.psql_con('WRITE') as con:
#     lr_out = pd.read_sql(f'''select out.*, sp.train_1, raw.risk_class
#                             from {schema}.{table} out
#                             inner join temp.hb_poc_splits sp
#                             on sp.contract_no=out.contract_no
#                             left join {decision_model_table} dm
#                             on dm.contract_no = out.contract_no
#                             left join 
#                                 (select distinct contract_no, risk_class from {lr_model_table}
#                                 where risk_class is not null and year_seq=5) raw
#                             on raw.contract_no = out.contract_no
#                             ''', con)

In [ ]:
# age_gp_interval = list(range(0, 56, 5)) + [np.inf]

# lr_out = (
#     lr_out
#     .query("risk_class == 'decline' or exposure==5 ")
#     .assign(age_gp=lambda lr_out: pd.cut(lr_out['app_age'], age_gp_interval, right=True, include_lowest=True))
#     .merge(df_lr_mapping[['contract_no','loss_ratio','tot_pay_amt', 'eff_annual_prem']], how='left', on='contract_no')
# )
# lr_out

In [ ]:
# lr_out = lr_out.replace({
#     'risk_class': {
#         'exclusion':'substandard', 
#         'exclusion_loading':'substandard', 
#         'loading':'substandard'
#         },
#             })
# lr_out_test = lr_out.query("train_1==0").copy()
# lr_out_train = lr_out.query("train_1==1").copy()

In [ ]:
# lr_out_test.bin_dlr

# lr_out_test.bin_dlr.value_counts()

# lr_out_test
# sns.relplot(data=lr_out_test, x='bin_dlr', y='bmi', kind='line', ci=None)

# sns.relplot(data='lr_out_test', x='bmi', y='bmi', kind='line')
# lr_out_test
# lr_out_test.groupby('bin_dlr').bmi.mean().plot()

# sns.relplot(data=lr_out_test.query("diff_high >0"), x='bin_dlr', y='bmi', kind='line', row='age_gp', col='sex_male', ci=None)


# g = sns.relplot(data=lr_out_test.query("clean_record==0"), x='bin_dlr', y='bmi', kind='line', row='age_gp', col='sex_male', ci=None, facet_kws={'sharex':False})


# sns.relplot(data=lr_out_test, x='bin_dlr', y='bmi', kind='line', row='age_gp', col='sex_male', ci=None, facet_kws={'sharex':False})
# sns.relplot(data=lr_out_test, x='bin_dlr', y='bmi', kind='scatter', row='age_gp', col='sex_male', ci=None)



# g = sns.relplot(data=lr_out_test, x='bin_dlr', y='bmi', kind='line', row='age_gp', col='sex_male', ci=None, facet_kws={'sharex':False})
# for ax in g.axes.flat:
#     _ = ax.set_xlabel('bin')
# g.tight_layout()

# plt.figure()
# sns.histplot(data=lr_out_test.query("diff_high>0"), x='bin_dlr', bins=20).set(title='bin distribution for those with diff_high>0')

# lr_out_test.query("diff_high>0").bin_dlr.plot.hist()